# Modelling distributions dependent on an entity class

In this notebook we will learn how to code simpy logic to mimic a process where there are two classes of arrivals to the system. Each arrival class has its own distribution, but requires the same resource.

We will work with a hypothetical walk-in health clinic where individual patients arrive with either minor trauma or non-trauma needs (the patient class). The patients wait in turn for a cubicle space, but their treatment time depends on their class. 

We will look at two ways to implement the logic.  

1. A single process with a dictionary lookup to select the distribution
2. A process per class type.

> In this example we will not concern ourselves with a warm-up period or initial conditions.

![model image](img/patient_classes.png "arrival classes")


## 1. Imports 

In [ ]:
import numpy as np
import itertools
import simpy

# we are going to use the built-in Enum to help with modelling patient class
from enum import Enum

In [ ]:
# to reduce code these classes can be found in distribution.py
from distributions import Exponential, Lognormal, Bernoulli

## 2. Constants

In [ ]:
# default mean inter-arrival times(exp)
# time is in minutes
IAT_WALK_IN = 8.0

# resources
N_CUBICLES = 2

# probability trauma
P_TRAUMA = 0.70

# minor trauma treatment time (Lognormal)
TRAUMA_TREAT_MEAN = 15.0
TRAUMA_TREAT_STD = 1.0

# non-trauma consultation time (Lognormal)
NONTRAUMA_CONSULT_MEAN = 5.0
NONTRAUMA_CONSULT_STD = 0.1

# sampling settings
N_STREAMS = 4
DEFAULT_RND_SET = 0

# Boolean switch to simulation results as the model runs
TRACE = False

# run variables (units = minutes)
RUN_LENGTH = 60.0 * 4

## 2. Helper classes and functions

In [ ]:
class PatientType(Enum):
    """A simple enumeration to discriminate between trauma and non-trauma.
    """
    NON_TRAUMA = 0
    TRAUMA = 1

In [ ]:
def trace(msg):
    """
    Turing printing of events on and off.

    Params:
    -------
    msg: str
        string to print to screen.
    """
    if TRACE:
        print(msg)

## 3. Experiment class

In [ ]:
class Experiment:
    """
    Encapsulates the concept of an experiment 🧪 for the stroke pathway
    bed blocking simulator. Manages parameters, PRNG streams and results.
    """
    def __init__(
        self,
        random_number_set=DEFAULT_RND_SET,
        n_streams=N_STREAMS,
        iat_walk_in=IAT_WALK_IN,
        p_trauma=P_TRAUMA,
        trauma_treat_mean=TRAUMA_TREAT_MEAN,
        trauma_treat_std=TRAUMA_TREAT_STD,
        nontrauma_consult_mean=NONTRAUMA_CONSULT_MEAN,
        nontrauma_consult_std=NONTRAUMA_CONSULT_MEAN,
        n_cubicles=N_CUBICLES
    ):
        """
        The init method sets up our defaults.
        """
        # sampling
        self.random_number_set = random_number_set
        self.n_streams = n_streams

        # store parameters for the run of the model
        self.iat_walk_in = iat_walk_in
        self.p_trauma = p_trauma
        self.trauma_treat_mean = trauma_treat_mean
        self.trauma_treat_std = trauma_treat_std
        self.nontrauma_consult_mean = nontrauma_consult_mean
        self.nontrauma_consult_std = nontrauma_consult_std

        #  place holder for resources
        self.cubicles = None
        self.n_cubicles = n_cubicles
        
        # initialise results to zero
        self.init_results_variables()

        # initialise sampling objects
        self.init_sampling()

    def set_random_no_set(self, random_number_set):
        """
        Controls the random sampling
        Parameters:
        ----------
        random_number_set: int
            Used to control the set of pseudo random numbers used by
            the distributions in the simulation.
        """
        self.random_number_set = random_number_set
        self.init_sampling()

    def init_sampling(self):
        """
        Create the distributions used by the model and initialise
        the random seeds of each.
        """
        # produce n non-overlapping streams
        seed_sequence = np.random.SeedSequence(self.random_number_set)
        self.seeds = seed_sequence.spawn(self.n_streams)

        # create distributions

        # inter-arrival time distributions
        self.arrivals = Exponential(
            self.iat_walk_in, random_seed=self.seeds[0]
        )

        self.patient_class = Bernoulli(self.p_trauma, self.seeds[1])
        
        # dictionary that will contain class lookup
        self.treatment_times = {}
        
        self.treatment_times[PatientType.TRAUMA] = Lognormal(
            self.trauma_treat_mean, self.trauma_treat_std, 
            random_seed=self.seeds[2]
        )

        self.treatment_times[PatientType.NON_TRAUMA] = Lognormal(
            self.nontrauma_consult_mean, self.nontrauma_consult_std, 
            random_seed=self.seeds[3]
        )

    def init_results_variables(self):
        """
        Initialise all of the experiment variables used in results
        collection.  This method is called at the start of each run
        of the model
        """
        # variable used to store results of experiment
        self.results = {}
        self.results["waiting_time"] = []
        # counts of arrivals
        self.results[PatientType.TRAUMA] = 0
        self.results[PatientType.NON_TRAUMA] = 0
        

In [ ]:
class Patient:
    """ Convenience class to store individual patient attributes
    """
    def __init__(self, unique_id: int, patient_type: PatientType):
        self.id = unique_id
        self.type = patient_type

    def __repr__(self) -> str:
        """string representation of patient for tracing"""
        return f"Patient {self.id} ({self.type.name})"

## 4. A single process for both classes

In [ ]:
def patient_pathway(patient, env, args):
    """Process a patient through the walk-in clinic. Treatment
    time depends on patient type.
    
    Parameters:
    -----------
    patient: Patient
        Container for patient information.
        
    env: simpy.Environment
        The simulation environment

    args: Experiment
        Container class for the simulation parameters/results.
    """
    arrival_time = env.now

    with args.cubicles.request() as cubicle_request:
        yield cubicle_request
        
        cubicle_assign_time = env.now
        wait_for_cubicle = cubicle_assign_time - arrival_time
        args.results['waiting_time'].append(wait_for_cubicle)
        
        trace(f"{env.now:.2f}: {patient} assigned cubicle." \
              + f"(waited {wait_for_cubicle:.2f} minutes)")
        
        # Simulate treatment/consult time depending on class
        # implemented as a dictionary lookup
        treat_duration = args.treatment_times[patient.type].sample()
    
        yield env.timeout(treat_duration)
        
        trace(f"{env.now:.2f}: {patient} exit")
        

## 4. Arrivals generator

This is a standard arrivals generator. We create stroke arrivals according to their distribution.

In [ ]:
def walk_in_generator(env, args):
    """
    Arrival process for walk in clinic.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_id in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.arrivals.sample()
        yield env.timeout(inter_arrival_time)

        # create a patient 
        patient = Patient(
            unique_id = patient_id, 
            patient_type = PatientType(args.patient_class.sample())
        )

        # increment count of patient arrivals for class
        args.results[patient.type] += 1
        
        trace(f"{env.now:.2f}: ARRIVAL - {patient}")

        # patient enters pathway
        env.process(patient_pathway(patient, env, args))

## 5. Single run function

In [ ]:
def single_run(
    experiment, 
    rep=0,
    run_length=RUN_LENGTH
):
    """
    Perform a single run of the model and return the results

    Parameters:
    -----------

    experiment: Experiment
        The experiment/paramaters to use with model

    rep: int
        The replication number.

    rc_period: float, optional (default=RUN_LENGTH)
        The run length of the model
    """

    # reset all results variables to zero and empty
    experiment.init_results_variables()

    # set random number set to the replication no.
    # this controls sampling for the run.
    experiment.set_random_no_set(rep)

    # environment is (re)created inside single run
    env = simpy.Environment()

    # simpy resources
    experiment.cubicles = simpy.Resource(env, experiment.n_cubicles)

    # we pass all arrival generators to simpy 
    env.process(walk_in_generator(env, experiment))

    # run model
    env.run(until=run_length)

    # quick stats
    results = {}
    results['mean_wait'] = np.array(
        experiment.results["waiting_time"]
    ).mean()
    
    results[PatientType.TRAUMA] = experiment.results[PatientType.TRAUMA]
    results[PatientType.NON_TRAUMA] = experiment.results[PatientType.NON_TRAUMA]


    # return single run results
    return results

In [ ]:
TRACE = True
experiment = Experiment()
results = single_run(experiment)
results

## Quick tests

### All patients are trauma

In [ ]:
TRACE = False
experiment = Experiment(p_trauma=1.0)
results = single_run(experiment)
results

### All patients are non-trauma

In [ ]:
TRACE = False
experiment = Experiment(p_trauma=0.0)
results = single_run(experiment)
results

## 5. A process per class

Alternatively you could implement process per arrival class. This is useful when process differ in more ways that just process duration. For example, there might be multiple steps or different resources used.  

To illustrate this we will add a second activity to the trauma pathway where the patients waits for 10 minutes before leaving.

To implement this we need two pathway processes 

* `trauma_pathway` a process for the trauma pathway (with ammended logic)
* `non_trauma_pathway` a process for the non-trauma pathway

We also need to ammend the `walk_in_generator` function to schedule the process dependent on class of arrival.

In [ ]:
def trauma_pathway(patient, env, args):
    """Process a TRAUMA patient through the walk-in clinic. 

    Ammended to include a second fixed duration activity before exit.
    
    Parameters:
    -----------
    patient: Patient
        Container for patient information.
        
    env: simpy.Environment
        The simulation environment

    args: Experiment
        Container class for the simulation parameters/results.
    """
    arrival_time = env.now

    with args.cubicles.request() as cubicle_request:
        yield cubicle_request
        
        cubicle_assign_time = env.now
        wait_for_cubicle = cubicle_assign_time - arrival_time
        args.results['waiting_time'].append(wait_for_cubicle)
        
        trace(f"{env.now:.2f}: {patient} assigned cubicle." \
              + f"(waited {wait_for_cubicle:.2f} minutes)")
        
        # MODIFICATION: here we always use PatientType.TRAUMA in the lookup.
        treat_duration = args.treatment_times[PatientType.TRAUMA].sample()
    
        yield env.timeout(treat_duration)

        trace(f"{env.now:.2f}: {patient} treatment complete.")

        # MODIFICATION: simulate variance from the non-trauma pathway
        yield env.timeout(10.0)
        
        trace(f"{env.now:.2f}: {patient} exit")
        

In [ ]:
def non_trauma_pathway(patient, env, args):
    """Process a NON_TRAUMA patient through the walk-in clinic. 
    
    Parameters:
    -----------
    patient: Patient
        Container for patient information.
        
    env: simpy.Environment
        The simulation environment

    args: Experiment
        Container class for the simulation parameters/results.
    """
    arrival_time = env.now

    with args.cubicles.request() as cubicle_request:
        yield cubicle_request
        
        cubicle_assign_time = env.now
        wait_for_cubicle = cubicle_assign_time - arrival_time
        args.results['waiting_time'].append(wait_for_cubicle)
        
        trace(f"{env.now:.2f}: {patient} assigned cubicle." \
              + f"(waited {wait_for_cubicle:.2f} minutes)")
        
        # MODIFICATION: here we always use PatientType.NON_TRAUMA in the lookup.
        treat_duration = args.treatment_times[PatientType.NON_TRAUMA].sample()
    
        yield env.timeout(treat_duration)
        
        trace(f"{env.now:.2f}: {patient} exit")

In [ ]:
def walk_in_generator(env, args):
    """
    Arrival process for walk in clinic.
    Modified to scheduled processes dependent on arrival class.

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for patient_id in itertools.count(start=1):

        # the sample distribution is defined by the experiment.
        inter_arrival_time = args.arrivals.sample()
        yield env.timeout(inter_arrival_time)

        # create a patient 
        patient = Patient(
            unique_id = patient_id, 
            patient_type = PatientType(args.patient_class.sample())
        )

        # increment count of patient arrivals for class
        args.results[patient.type] += 1
        
        trace(f"{env.now:.2f}: ARRIVAL - {patient}")

        if patient.type is PatientType.TRAUMA:
            # patient enters trauma pathway
            env.process(trauma_pathway(patient, env, args))
        else:
            # patient enters non-trauma pathway
            env.process(non_trauma_pathway(patient, env, args))

In [ ]:
TRACE = True
experiment = Experiment()
results = single_run(experiment)
results

### Quick test!

In [ ]:
TRACE = False
experiment = Experiment(p_trauma=1.0)
results = single_run(experiment)
results